# Initialize

In [1]:
import pandas as pd
import numpy as np
import altair as alt

alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

# Load Data

In [2]:
df = pd.read_csv("../data/casestudy.csv", index_col = [0])
yeardf = pd.DataFrame(index=df['year'].unique(), 
                      columns=['Total Revenue', 
                               'New Customer Revenue',
                               'Existing Customer Growth',
                               'Revenue Loss from Attrition',
                               'Existing Customer Revenue Current Year',
                               'Existing Customer Revenue Prior Year',
                               'Total Customers Current Year',
                               'Total Customers Previous Year',
                               'New Customers',
                               'Lost Customers'])
pivot = df.pivot(index='customer_email', columns='year', values='net_revenue')

num_years = df['year'].unique().size

In [4]:
pivot_matrix = pivot.to_numpy(copy=True)
transpose_matrix = pivot_matrix[::-1, ::-1].T

prev = 0
rows = transpose_matrix.shape[0]
cols = transpose_matrix.shape[1]

for i in range(1, rows):
    for j in range(0, cols):
        if np.isnan(transpose_matrix[prev][j]):
            continue
        else:
            if not np.isnan(transpose_matrix[i][j]):
                transpose_matrix[prev][j] = np.nan
    prev = i

pivot_matrix = transpose_matrix[::-1, ::-1].T
newcustdf = pd.DataFrame()

for i in range(num_years):
    title = 2015 + i
    newcustdf.insert(len(newcustdf.columns), title, pivot_matrix[:, i])

yeardf['Total Revenue'] = pivot.sum()
yeardf['New Customer Revenue'] = newcustdf.sum()
yeardf['Total Customers Current Year'] = pivot.count()
yeardf['Total Customers Previous Year'] = yeardf['Total Customers Current Year'].shift(1)
yeardf['New Customers'] = newcustdf.count()

In [5]:

for i in range(1, num_years):
    currkey = 2015 + i
    prevkey = currkey - 1
    dfslice = pivot[pivot[prevkey].notna() & pivot[currkey].notna()]
    revenues = dfslice.sum()
    customers = dfslice.count()
    yeardf.loc[currkey, 'Existing Customer Growth'] = revenues[currkey] - revenues[prevkey]
    yeardf.loc[currkey, 'Revenue Loss from Attrition'] = yeardf.loc[prevkey, 'Total Revenue'] - revenues[currkey]
    yeardf.loc[currkey, 'Existing Customer Revenue Current Year'] = revenues[currkey]
    yeardf.loc[currkey, 'Existing Customer Revenue Prior Year'] = revenues[prevkey]
    yeardf.loc[currkey, 'Lost Customers'] = yeardf.loc[prevkey, 'Total Customers Current Year'] - customers[currkey]

In [6]:
yeardf

,Total Revenue,New Customer Revenue,Existing Customer Growth,Revenue Loss from Attrition,Existing Customer Revenue Current Year,Existing Customer Revenue Prior Year,Total Customers Current Year,Total Customers Previous Year,New Customers,Lost Customers
2015,29036749.19,29036749.19,NaN,NaN,NaN,NaN,231294,NaN,231294,NaN
2016,25730943.59,18245491.01,20335.46,21551296.61,7485452.58,7465117.12,204646,231294.0,145062,171710
2017,31417495.03,28776235.04,20611.34,23089683.6,2641259.99,2620648.65,249987,204646.0,229028,183687
